### 구글드라이브 연계

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


★ 설정부분: 본인의 학습데이터가 저장되어 있는 폴더경로를 설정하세요


예: "/content/gdrive/MyDrive/Colab Notebooks/llama2_custom/dataset/"


In [7]:
dataPath = "/content/gdrive/MyDrive/Colab Notebooks/llama2_custom/dataset/"

### 라이브러리 설치

In [5]:
# datasets: hugging-face의 인공지능 커뮤니티의 데이터를 불러오고 관리하는 라이브러리
# 기존 라이브러리가 2월19일 기준 업데이트 되면서 pyarrow 라이브러리와 호환성 문제 발생 띠리사 호환라이브러리 정의
# jsonlines: json 포맷의 데이터를 핸들링하는 라이브러리
!pip install datasets==2.16.1 jsonlines==4.0.0

### huggign face 접근을 위한 로그인

In [6]:
# hugging face 로그인: hugging face -> 우측상단의 계정선택 -> settings -> access token
import huggingface_hub
huggingface_hub.login()

### 설치한 라이브러리 선언

In [8]:
# 행과열을 핸들링하는 라이브러리
import pandas as pd
# json 포맷 데이터를 핸들링하는 라이브러리
import json
import jsonlines
# hugging face 데이터 관리 라이브러리
from datasets import Dataset

1. 학습 데이터 불러오기 및 변환

★ 설정부분: 본인의 학습데이터 파일명을 설정하세요


예: "/content/gdrive/MyDrive/Colab Notebooks/llama2_custom/dataset/" 경로 내 indata_kor.csv 파일이면 indata_kor.csv

jsonFileName은 그냥 동일이름의 .jsonl이면 됨


In [9]:
# 데이터 경로 설정 및 불러오기
datasetName = "indata_kor.csv"
jsonFileName = "indata_kor.jsonl"

In [12]:
def csv_to_json(csv_file_path, json_file_path):
    # CSV 파일을 DataFrame으로 읽기★ (한글인 경우 encoding="ms949") 영문인경우 해당 내용 삭제 ★
    df = pd.read_csv(csv_file_path, encoding="ms949")

    # JSON 파일로 저장
    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        # 각 행을 JSON으로 변환하여 바로 파일에 쓰기
        for index, row in df.iterrows():
            data = {'inputs': row['inputs'], 'response': row['response']}
            json.dump(data, json_file, ensure_ascii=False)
            json_file.write('\n')  # 각 행마다 줄바꿈

# CSV 파일 경로와 JSON 파일 경로 설정
csv_file_path = dataPath + datasetName
json_file_path = dataPath + jsonFileName

# 함수 호출
csv_to_json(csv_file_path, json_file_path)

2. 파인튜닝용 포맷으로 변환

In [13]:
indataset = []
with jsonlines.open(json_file_path) as f:
    for line in f.iter():
      indataset.append(f'<s>[INST] {line["inputs"]} [/INST] {line["response"]} </s>')
    #   indataset.append(f'<s>### Instruction: \n{line["inputs"]} \n\n### Response: \n{line["response"]}</s>')

# 데이터셋 확인
print('데이터셋 확인')
print(indataset[:5])

# 데이터셋 생성 및 저장
indataset = Dataset.from_dict({"text": indataset})
indataset.save_to_disk(dataPath)

# 데이터셋 info 확인
print('데이터셋 info 확인')
print(indataset)

데이터셋 확인
['<s>[INST] 유튜브 채널 hkcode에서는 무엇을 가르치나요? [/INST] 초보자 대상으로 빅데이터, 인공지능과 관련된 컨텐츠를 가르치고 있습니다. </s>', '<s>[INST] 유튜브 채널 hkcode는 누가 운영하나요? [/INST] 한국폴리텍대학 스마트금융과 김효관 교수가 운영합니다. </s>', '<s>[INST] 스마트금융과는 무엇을 가르치나요? [/INST] 스마트금융과는 빅데이터, 인공지능, 웹개발 및 블록체인을 가르치고 있습니다. </s>', '<s>[INST] 스마트금융과 등록비용은 얼마인가요? [/INST] 등록비용은 국비지원 과정으로 무료 입니다. </s>', '<s>[INST] 스마트금융과는 1년에 몇 명을 선발하나요? [/INST] 1년에 한반을 운영하고 있고 최대 27명을 선발합니다. </s>']


Saving the dataset (0/1 shards):   0%|          | 0/32 [00:00<?, ? examples/s]

데이터셋 info 확인
Dataset({
    features: ['text'],
    num_rows: 32
})


3. huggingface 에 데이터 업로드 (★ 변경 포인트!)

In [14]:
indataset.push_to_hub("hyokwan/hkcode_korea")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/hyokwan/hkcode_korea/commit/ea080f64085c794ad7235755f7011535d2bd0f24', commit_message='Upload dataset', commit_description='', oid='ea080f64085c794ad7235755f7011535d2bd0f24', pr_url=None, pr_revision=None, pr_num=None)